## Assignment 11

Name: Kesav Adithya Venkidusamy<br>
Course: DSC650 - Big Data<br>
Instructor: Amirfarrokh Iranitalab<br>

Using section 8.1 in Deep Learning with Python as a guide, implement an LSTM text generator. Train the model on the Enron corpus or a text source of your choice. Save the model and generate 20 examples to the results directory of `dsc650/assignments/assignment11/`.

#### Input Data

In [1]:
## Get the input data
## Reference: Deep learning with Python

import keras
import numpy as np
path = keras.utils.get_file(
'nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 1us/step
Corpus length: 600893


#### Vectorize the text

In [2]:
# Vectorizing sequences of characters
maxlen = 60     # Extract sequences of 60 characters

step = 3        # Sample a new sequence every three characters

sentences = []  # Holds extracted sequences

next_chars = [] # Holds targets (the follow-up characters)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text))) 
print('Unique characters:', len(chars))
# Dict that maps unique characters to their index in the list `chars`
char_indices = dict((char, chars.index(char)) for char in chars) 

print('Vectorization...')
# One-hot encodes the caracters into binary arrays:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


#### Model Setup

In [5]:
# Single-layer LSTM model for next-character prediction
# load libraries
from keras import layers
import tensorflow as tf

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [6]:
# Model compilation configuration
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
# Function to sample the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#### Text Generation Loop 

In [17]:
import random
import sys
from pathlib import Path

inum = 1
results_dir = Path("/home/jovyan/dsc650/dsc650/assignments/assignment11")

for epoch in range(1, 21):
    print('\nepoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print('\n--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n----- temperature:', temperature)
        print("start text")
        sys.stdout.write(generated_text)
        print("\nend text")
        print("**************Loop write start*************")
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
        print("\n**************Loop write end**************")
        
        print("\nPrinting the Generated Text for the temperature {}".format(temperature))
        print(generated_text)

        if inum <= 21 and temperature in [1, 1.2]:
            print("\nWriting the generated text for the temperature {} to file: {}".format(temperature, inum))
            output_file = 'LSTM_Text_Generated_Temperature_'+ str(temperature) +'_EPOCH_'+str(epoch)+'_Sample_' + str(inum) + '.txt'
            result_model_file = results_dir.joinpath(output_file)
            f = open(result_model_file, "w")
            f.write(generated_text)
            f.close()
            inum+= 1


epoch 1
1565/1565 [==============================] - 200s 128ms/step - loss: 1.2900

--- Generating with seed: "s, an immense physiological process goes on, which is ever
e"

----- temperature: 0.2
start text
s, an immense physiological process goes on, which is ever
e
end text
**************Loop write start*************
nough and almost of the spiritual profound to the spiritual problem of the most consideration of the spiritual and the sense of the present and the sense of the conceptions of the spiritual and all the most consideration of the spiritual and strong and the proposition of the spiritual and the most consideration of the art of the spiritual and all the proper and the sense of the spiritual and the s
**************Loop write end**************

Printing the Generated Text for the temperature 0.2
 and all the proper and the sense of the spiritual and the s

----- temperature: 0.5
start text
 and all the proper and the sense of the spiritual and the s
end text
*************

<ipython-input-7-d96798082d66>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 go the man of the spiritual so experience itself, the fact and something be stand times of the conceptions of the belief by a states to the sense and this
age the various first and almost and at present getter, that they gerxen and relation of the most soming of the way is also preparations of the sense of the faith and so say there are themselves by the most indiscivion
**************Loop write end**************

Printing the Generated Text for the temperature 0.5
aith and so say there are themselves by the most indiscivion

----- temperature: 1.0
start text
aith and so say there are themselves by the most indiscivion
end text
**************Loop write start*************

in which
its! woman an
virtual we inselfing the
jout for as its proposition is brought though according to alpoverance,
and well believe exembisest. thereinus , all literatures. in functier; for in the stattem as horars as e destion and shimit and other, really, a senies, posit morality. and as a wiallo's, that which

1565/1565 [==============================] - 200s 128ms/step - loss: 1.2840

--- Generating with seed: "ay by such a
universal rule of conduct, it behooves it to at"

----- temperature: 0.2
start text
ay by such a
universal rule of conduct, it behooves it to at
end text
**************Loop write start*************
tain the profound themselves the conscience of the sense and the profound more the more the spirits and the sense of the same and the conscience of the sense of the spirit, and the presence of the more the sense of the spirit, and the sense of the conscience of the scientific spirits, the least and self-contrary and such a sense of the spirit, in the belief in the sense of the sense of the conscie
**************Loop write end**************

Printing the Generated Text for the temperature 0.2
irit, in the belief in the sense of the sense of the conscie

----- temperature: 0.5
start text
irit, in the belief in the sense of the sense of the conscie
end text
**************Loop wri

any two tcan silence, begined. we are senset of an avestening thereby would may be slous a mistaken for considered! altheed and grehapoundan, among more imparery, invanates is a conceantly circquately on europe, in the fearful delication of the bravery orage of last saga, for hiced to "the act of
actory of themselves in even principled
crabelierpe justification. here the debel
man--thing is fasma,
**************Loop write end**************

Printing the Generated Text for the temperature 1.0
abelierpe justification. here the debel
man--thing is fasma,

Writing the generated text for the temperature 1.0 to file: 11

----- temperature: 1.2
start text
abelierpe justification. here the debel
man--thing is fasma,
end text
**************Loop write start*************
 amongorism anyoutmously a belief for the fearving fortuness cuward
into woman things to bood oul--namely. prevenge undiblietness as sy: to eolity: it is manifely, when they consider be
truth end andrerness traaks to less circest

lighteness of the spirit of the sense of the sentiment of the spirit and the spirit of the state of the spirit, and the state of the senses of the state of the state of the sentence of the commanding the strength of the commanding the most christian and soul and the state of the sentence, and the sentiment of the still
soul is the state of the state of the senses and alless of the same men and the
**************Loop write end**************

Printing the Generated Text for the temperature 0.2
f the state of the senses and alless of the same men and the

----- temperature: 0.5
start text
f the state of the senses and alless of the same men and the
end text
**************Loop write start*************
 intellect of a philosopher which every man, a philosopher, which the feeling mistakely to have been the most power that of the stranges of all the faith as the sentence, as the scorn and bread are interest as storiting, and now himself which he seems to be necessary and the artistical thinke

for poi ofscky;
"wylal
plausupals as i shor-dream that it may be min. "on the wafffuliry
collance!--but whom manifest akques.
so conscience,--fines music fucian into many a science,--the expetinable upon this way spirits: but there the powerful. the
profound, simitands
to scoied thirs
pride it should
natur
s, whoever dogmans pare that which wimen stee vernery thus habit          accelitally distru
**************Loop write end**************

Printing the Generated Text for the temperature 1.2
ch wimen stee vernery thus habit          accelitally distru

epoch 12
1565/1565 [==============================] - 200s 128ms/step - loss: 1.2749

--- Generating with seed: "y more--except the theologians themselves? apart from all
th"

----- temperature: 0.2
start text
y more--except the theologians themselves? apart from all
th
end text
**************Loop write start*************
e sense of the spirit and the spirit than the sense and stronger and spiritual conscious the most surpreding in the s

ry conception in name.

4him hood tack these middans poppreterion; he do nood that all will cause at under man.-mlantess. a relly mied
and to himself of the age of moralitiesnive
belong" and trif of the erron of the sembises indisfulled the motive,
who communitners compares of
consiated: but with religious and final and around
sha!ly mecome, at onber
surposely at ong and creating as ike in the
and
**************Loop write end**************

Printing the Generated Text for the temperature 1.0
me, at onber
surposely at ong and creating as ike in the
and

----- temperature: 1.2
start text
me, at onber
surposely at ong and creating as ike in the
and
end text
**************Loop write start*************
ic--these beauticn"
vitient ipport in name, or schention than
it, ever
enough it-day of grotits.ned, as should with the
peoulsticul ?limes      on athoused .thatsow, or us
theist bryon these mul.nide. het yet ethics, as far negereable fals sympathy, the vain, novelty of
course--ashess!

al an

                                                                                                                                                                                                                                                                                                                                                                                                                
**************Loop write end**************

Printing the Generated Text for the temperature 0.5
                                                            

----- temperature: 1.0
start text
                                                            
end text
**************Loop write start*************
     
of live--one must afcound; that is a their farirtic stronger knet were something is is has flow. in
the hypethrowed the point and course, the usefice strict him senses roncipulated to believe how
gregered ffefities.
but
only in favour as new place is the sentence, sociest.
-zove, or far

s the conception of the strong and more and all the spirit and soul, as the sense of the world and constituted the stronger and spirit and profoundly of the profound moral party of the spirit of the spirit and the astons there is a sort of things as the profound the strong and all the past and belief in the senses of the most artists of the strongest for the strength of the fact that the senses of
**************Loop write end**************

Printing the Generated Text for the temperature 0.2
he strongest for the strength of the fact that the senses of

----- temperature: 0.5
start text
he strongest for the strength of the fact that the senses of
end text
**************Loop write start*************
 the last pasting believe therein--in the socially with the consequence, and the strongly and the greatest hand
and such a pery hand, and the past of the present conception of the seldom of exulted and more wars of the former it as unintermined patien dear and who has not be
the best man is p